<a href="https://colab.research.google.com/github/ngarciap/AnalisisExploratorioDeDatos/blob/main/Exploraci%C3%B3n_y_limpieza_de_archivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Cargue de archivos de Rxs del Mercado Farmacéutico Colombiano**
***Preparación y Cargue de un Dataset / Asociado a mi pregunta SMART***

In [ ]:
# Importar librerias
import pandas as pd
from google.colab import drive

# Cargar Dataframes
drive.mount('/content/drive', force_remount=True)
df_producto = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/CUP_PRODUTO.TXT', sep='	', encoding='latin1', dtype=str)
df_laboratorio = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/CUP_LABORATORIO.TXT', sep='	', encoding='latin1', dtype=str)
df_prescripciones = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Analisis Exploratorio de Datos/Bases/Market COL Jun/CUP_FATO.TXT', sep='	', encoding='latin1', dtype=str)

Mounted at /content/drive


**Se procede a Cruzar la Tabla principal de Rx con Productos y Laboratorios para poder obtener un Dataframe que me inidque prescripciones por laboratorio del último mes**

In [ ]:
#Alistamiento de tabla de productos para traer el Laboratorio
df_producto_completo = pd.merge(df_producto, df_laboratorio, how='inner', left_on='PRO_LAB_ID', right_on='LAB_ID')

#Alistamiento de tabla de prescripciones para la columna a Analizar
df_prescripciones['MES00'] = df_prescripciones['MES00'].astype(int)

# Cruce de Prescrupciones con Producto y filtrado de Columnas de interés
df_prescripciones = pd.merge(df_prescripciones, df_producto_completo, how='left', on='PRO_ID')
#df_analizar = df_prescripciones.groupby('LAB_DESC')['MES00'].sum().reset_index()
df_analizar = df_prescripciones[['LAB_DESC','MES00']]

**Se realiza análisis sobre el Dataframe que contiene las prescripciones por laboratorio**

In [ ]:
df_analizar.head()

,LAB_DESC,MES00
0,BOYDORR,0
1,BOYDORR,0
2,BOYDORR,0
3,BOYDORR,0
4,BOYDORR,0


In [ ]:
df_analizar.describe()

,MES00
count,1.753165e+06
mean,3.074702e-01
std,2.632017e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,8.170000e+02


Se observa que en promedio, hay 3 Rx por producto,  y que el el valor de Rx máximo es de 817 en un mes.

In [ ]:
df_analizar.LAB_DESC.value_counts()

,count
LAB_DESC,
LAB NO INDICADO,384083
MCKESSON,102273
TECNOQUIMICAS,65116
LAFRANCOL,62928
MEGALABS MLB,50353
...,...
AZ PHARMA,1
BLEMOX,1
GENERICS GL,1


En un mes se da que la mayoría de Rx en un mes no se especifica el laboratorio, adicional que los 2 laboratorios principales con más Rx son MK y TQ seguidos por Lafrancol.